In [15]:
import mne 
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
figure(figsize=(1,1))
from mne.decoding import UnsupervisedSpatialFilter
from sklearn.decomposition import PCA, FastICA
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import make_pipeline
from mne.decoding import Vectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
import itertools

%run ../capstone2/Untitled.ipynb

In [2]:
def arrayFiles(path):
    folderContent = os.listdir(path)
    fifFiles = []
    for i in folderContent :
        fifFiles.append(path + "/" + i)
    
    return fifFiles

In [3]:
#concatenate path to name of each file and store in array for reading in data
path = '/Volumes/flash64/EpochedEEG'
allFifFiles = arrayFiles(path)

In [4]:
epoch = mne.read_epochs(allFifFiles[0],preload=True)

Reading /Volumes/flash64/EpochedEEG/20131216_1441_epo.fif ...
    Found the data of interest:
        t =    -199.22 ...     500.00 ms
        0 CTF compensation matrices available
5424 matching events found
Applying baseline correction (mode: mean)
5424 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated


In [5]:
filtered_epoch = epoch[(epoch.events[:,-1] >= 1) & (epoch.events[:,-1] <= 24)]

In [6]:
filtered_epoch.event_id

{'flanker_stim_1': 1,
 'flanker_stim_10': 10,
 'flanker_stim_11': 11,
 'flanker_stim_12': 12,
 'flanker_stim_2': 2,
 'flanker_stim_3': 3,
 'flanker_stim_4': 4,
 'flanker_stim_5': 5,
 'flanker_stim_6': 6,
 'flanker_stim_7': 7,
 'flanker_stim_8': 8,
 'flanker_stim_9': 9,
 'target_stim_13': 13,
 'target_stim_14': 14,
 'target_stim_15': 15,
 'target_stim_16': 16,
 'target_stim_17': 17,
 'target_stim_18': 18,
 'target_stim_19': 19,
 'target_stim_20': 20,
 'target_stim_21': 21,
 'target_stim_22': 22,
 'target_stim_23': 23,
 'target_stim_24': 24}

In [16]:
modal_lexi

{'AALL': [(5, 6, 4), (17, 18, 16)],
 'AALN': [(11, 6, 10), (23, 18, 22)],
 'AANL': [(5, 12, 4), (17, 24, 16)],
 'AANN': [(11, 12, 10), (23, 24, 22)],
 'AVLL': [(2, 6, 1), (14, 18, 13)],
 'AVNN': [(8, 12, 7), (20, 24, 19)],
 'VALL': [(5, 3, 4), (17, 15, 16)],
 'VANN': [(11, 9, 10), (23, 21, 22)],
 'VVLL': [(2, 3, 1), (14, 15, 13)],
 'VVLN': [(8, 3, 7), (20, 15, 19)],
 'VVNL': [(2, 9, 1), (14, 21, 13)],
 'VVNN': [(8, 9, 7), (20, 21, 19)]}

In [17]:
#0 for visual and 1 for audio
test = filtered_epoch

for i in range(0,1280):
    if test.events[i][-1] in (7,8, 9, 19, 20, 21):
        test.events[i][-1] = 0
    elif test.events[i][-1] in (10, 11, 12, 22, 23, 24):
        test.events[i][-1] = 1

In [20]:
test.events

array([[  2593,      0,      0],
       [  2718,      0,      3],
       [  2847,      0,      0],
       ...,
       [658626,      0,      2],
       [658755,      0,      9],
       [658880,      0,      1]])

In [21]:
dataXX = test.get_data()
eventsYY = test.events[:,-1]

clf = make_pipeline(Vectorizer(),
                    MinMaxScaler(),
                    LDA())

#cross validation
cv = StratifiedKFold(n_splits = 10, shuffle=True)
preds = np.empty(len(eventsYY))
scoresss = []
for train,test in cv.split(dataXX,eventsYY):
    clf.fit(dataXX[train],eventsYY[train])
    preds[test] = clf.predict(dataXX[test])
    scoresss.append(clf.score(dataXX[test],eventsYY[test]))

/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserW

In [22]:
print("mean classification score", np.mean(scoresss))
report = classification_report(eventsYY, preds, target_names=target_names)
print(report)

mean classification score 0.23537926761628197


NameError: name 'target_names' is not defined